In [2]:
from sklearn.linear_model import SGDClassifier
import numpy as np
import collections
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import confusion_matrix
from scipy.special import logsumexp


import itertools
from collections import defaultdict
import collections

from functools import reduce

from multiprocess import Pool
from functools import partial
import numpy as np

import itertools 
from itertools import islice
import games

import cpb
import bpm
import random_algo

import plotly.graph_objects as go
import TSPM

import cpb_gaussian
import PM_DMED

# import feedexp3_bianchi
import feedexp3_piccolboni


def evaluate_parallel(nbCores, n_folds, horizon, alg, game, type):

    print("nbCores:", nbCores, "nbFolds:", n_folds, "Horizon:", horizon)
    pool = Pool(processes = nbCores) 
    task = Evaluation(horizon, type)

    np.random.seed(1)
    distributions = []

    for jobid in range(n_folds):
        
        if type == 'easy' :
            p = np.random.uniform(0, 0.2) #if np.random.uniform(0,1)>0.5 else np.random.uniform(0.8, 1)
        #elif type == 'easy' and jobid > 100:
        #    p = np.random.uniform(0.8, 1)
        else:
            p = np.random.uniform(0.4,0.6)
        distributions.append( [p, 1-p] )

    return np.asarray(  pool.map( partial( task.eval_policy_once, alg, game ), zip(distributions ,range(n_folds)) ) ) 

class Evaluation:

    def __init__(self, horizon, type ):
        self.horizon = horizon
        self.type = type
        # self.outcome_distribution = outcome_distribution

    def get_outcomes(self, game, job_id):
        # self.means = runif_in_simplex( self.game.n_outcomes )
        outcomes = np.random.choice( game.n_outcomes , p= list( game.outcome_dist.values() ), size= self.horizon) 
        return outcomes

    def get_feedback(self, game, action, outcome):
        return game.FeedbackMatrix[ action ][ outcome ]

    def eval_policy_once(self, alg, game, job):

        alg.reset()

        distribution, jobid = job

        #print('seed {} distribution {}'.format(jobid, distribution)) 
        np.random.seed(jobid)
        
        # outcome_distribution =  {'spam':0.05,'ham':0.95}

        outcome_distribution = {'spam':distribution[0],'ham':distribution[1]}

        # p = get_easy() if game.mode == 'easy' else get_harsch() 
        # outcome_distribution =  {'a':p[0],'b':p[1],'c':p[2],'d':p[3],'e':p[4]}

        game.set_outcome_distribution( outcome_distribution, jobid )
        optimal_action = game.i_star

        action_counter = np.zeros( (game.n_actions, self.horizon) )
        cum_regret = []

        # generate outcomes obliviously
        outcomes = self.get_outcomes(game, jobid)
        # outcomes, summary = self.distribution_shift(game )

        for t in range(self.horizon):

            # policy chooses one action
            action = alg.get_action(t)

            # Environment chooses one outcome
            outcome = outcomes[t]

            #print('t', t, 'action', action, 'outcome', outcome, )

            feedback =  self.get_feedback( game, action, outcome )

            alg.update(action, feedback, outcome, None, t)
            
            # print('nu', alg.nu / alg.n )

            for i in range(game.n_actions):
                if i == action:
                    action_counter[i][t] = action_counter[i][t-1] + 1
                else:
                    
                    action_counter[i][t] = action_counter[i][t-1]

        # regret = []
        # for t in range(self.horizon):
        #     regret.append(  self.delta_t( game, summary, t )  @ action_counter[:,t] )
            # cum_regret.append(  game.LossMatrix[action,outcome] - min( game.LossMatrix[...,outcome ] )  )
            
        regret = np.array( [ game.delta(i) for i in range(game.n_actions) ]).T @ action_counter
        
        return regret #np.cumsum(cum_regret)

In [ ]:
n_cores = 16
n_folds = 100
horizon = 100000

game = games.label_efficient()

algos = [ random_algo.Random(  game, horizon, ), 
          cpb.CPB(  game, horizon, 1.01),  
          PM_DMED.PM_DMED(  game, horizon, 100), 
          PM_DMED.PM_DMED(  game, horizon, 10), 
          PM_DMED.PM_DMED(  game, horizon, 5), 
          PM_DMED.PM_DMED(  game, horizon, 1),
          PM_DMED.PM_DMED(  game, horizon, 0.1),  
          TSPM.TSPM_alg(  game, horizon, 1),
          TSPM.TSPM_alg(  game, horizon, 0),
          bpm.BPM(game, horizon) ]

colors = [  [0,0,0],  [250,0,0], [0,250,0], [0,150,0], [0,0,250], [0,0,200], [0,0,150], [0,0,100],  [225,0,225], [150 , 0, 150] ] 
labels = [   'random', 'CBP',  'PM_DMEDc100', 'PM_DMEDc10', 'PM_DMEDc5', 'PM_DMEDc1', 'PM_DMEDc01', 'TSPM_R1', 'TSPM_R0', 'BPM_LEAST'  ]  

fig = go.Figure(    )

for alg, color, label in zip( algos, colors, labels):

    result = evaluate_parallel(n_cores, n_folds, horizon, alg, game, 'easy')
    np.save('./results/label_efficient/easy_{}_{}_{}'.format(horizon,n_folds, label), result)

    result = evaluate_parallel(n_cores, n_folds, horizon, alg, game, 'hard')
    np.save('./results/label_efficient/hard_{}_{}_{}'.format(horizon,n_folds, label), result)

In [ ]:


n_cores = 1
n_folds = 1
horizon = 100

# np.seterr(all='raise')

# game = games.apple_tasting(False, outcome_distribution) 



outcome_distribution = [0.1077633468006714, 0.8922366531993287]
job = (outcome_distribution, 9 )

game =  games.label_efficient(  ) 
game.set_outcome_distribution( {'spam':outcome_distribution[0],'ham':outcome_distribution[1]} )
print('optimal action', game.i_star)

# print('optimal action', game.i_star)
alg = PM_DMED.PM_DMED(  game, horizon,) #cpb.CPB(  game, horizon,1.01) #TSPM.TSPM_alg(  game, horizon, 1)
task = Evaluation(horizon, 'easy')

result = task.eval_policy_once(alg,game, job)
#plt.plot(range(horizon), result)
# fig = go.Figure( )
# regret = np.array([ game.delta(i) for i in range(game.n_actions) ]).T @ np.mean(result,0) 
# xcoords = np.arange(0,horizon,1).tolist()

# fig.add_trace(go.Scatter(x=xcoords, y=regret, line=dict(color='blue'), mode='lines',  name='TPSM' )) # 


In [ ]:


n_cores = 16
n_folds = 100
horizon = 100000

game = games.label_efficient(  )

#feedexp3.FeedExp3(  game, horizon, ),
#feedexp3_v3.FeedExp3(  game, horizon, ),
#eTSPM.eTSPM_alg(  game, horizon, 1),
#TSPM.TSPM_alg(  game, horizon,), bpm.BPM(  game, horizon,  [0.5, 0.5 ], np.identity(2) ) ,

#TSPM.TSPM_alg(  game, horizon,), 
#ucbTSPM_v2.TSPM_alg(game, horizon)  

algos = [ random_algo.Random(  game, horizon, ), 
          cpb.CPB(  game, horizon, 1.01), 
          None, 
          PM_DMED.PM_DMED(  game, horizon, 100), 
          PM_DMED.PM_DMED(  game, horizon, 10), 
          TSPM.TSPM_alg(  game, horizon, 1),
          TSPM.TSPM_alg(  game, horizon, 0),
          bpm.BPM(game, horizon) ]

colors = [  [0,0,0],  [250,0,0], [0,250,0], [255,128,0], [255,128,0],  [0 , 250, 250],  [255 , 0, 255] ] 
labels = [   'random', 'CBP', 'RandCBP', 'PM_DMEDc100', 'PM_DMEDc10', 'TSPM_R1', 'BPM_LEAST'  ]  

fig = go.Figure( )

final_regrets = []


for alg, color, label in list(zip(algos, colors, labels))[::-1]:

    r,g,b = color
    # result = evaluate_parallel(n_cores, n_folds, horizon, alg, game, 'harsch')
    # np.save('./results/label_efficient/hard_{}_{}_{}'.format(horizon,n_folds, label), result)
    
    if label == 'RandCBP':
        result = np.load( './results/benchmark_randcbp/label_efficient/LE_hard_{}_{}_Gaussian_1_10_10e7.npy'.format(horizon,n_folds, label) )
    else: 
        result = np.load( './results/label_efficient/hard_{}_{}_{}.npy'.format(horizon,n_folds, label) )

    final_regrets.append( result[:,-1] )
    regret =  np.mean(result, 0) 
    xcoords = np.arange(0,horizon,1).tolist()
    std =  np.std(result,0) 
    upper_regret = regret + std

    if label == 'PM_DMEDc100':
        fig.add_trace(go.Scatter(x=xcoords, y=regret, line=dict(color='rgb({},{},{})'.format(r,g,b),  dash=  'solid' ), mode='lines',  name='PMDMED (c=100)' ))  

    elif label == 'PM_DMEDc10':
        fig.add_trace(go.Scatter(x=xcoords, y=regret, line=dict(color='rgb({},{},{})'.format(r,g,b),  dash=  'longdash' ), mode='lines',  name='PMDMED (c=10)' )) 

    elif label == 'TSPM_R1':
        fig.add_trace(go.Scatter(x=xcoords, y=regret, line=dict(color='rgb({},{},{})'.format(r,g,b),  dash=  'solid' ), mode='lines',  name='TSPM (R=1)' )) 

    elif label == 'TSPM_R0':
        fig.add_trace(go.Scatter(x=xcoords, y=regret, line=dict(color='rgb({},{},{})'.format(r,g,b),  dash=  'dot' ), mode='lines',  name='TSPM (R=0)' )) 

    else:

        fig.add_trace(go.Scatter(x=xcoords, y=regret, line=dict(color='rgb({},{},{})'.format(r,g,b),  dash=  'solid' ), mode='lines',  name=label )) # 

    if label == 'PM_DMEDc100':
        fig.add_trace(   go.Scatter( x=xcoords+xcoords[::-1], y=upper_regret.tolist()+regret.tolist()[::-1],  fill='toself', fillcolor='rgba({},{},{},0.4)'.format(r,g,b), 
                         line=dict(color='rgba(255,255,255,0)'),   hoverinfo="skip",  showlegend=False )  )
    else:
        fig.add_trace(   go.Scatter( x=xcoords+xcoords[::-1], y=upper_regret.tolist()+regret.tolist()[::-1],  fill='toself', fillcolor='rgba({},{},{},0.2)'.format(r,g,b), 
                         line=dict(color='rgba(255,255,255,0)'),   hoverinfo="skip",  showlegend=False )  )
    
fig.update_layout(legend= dict(yanchor="top",y=0.98,xanchor="left",x=0.1), autosize=False,
                  xaxis_title="Sequence", yaxis_title="Regret",  margin=go.layout.Margin( l=0,   r=0,   b=0,    t=0, ),   font=dict(size=13,) )

fig.update_yaxes( type="log",range=[0, 5] )
fig.write_image("./hard_LE.pdf")

fig.update_xaxes( type="log" )
fig.write_image("./hard_LE_log.pdf")

fig.show()

final_regrets = np.array(final_regrets)
final = [ ( np.argmin(final_regrets[:,i]), i) for i in range(n_folds) ]

In [34]:
### HARD LABEL EFFICIENT:

labels = [  'CBP', 'RandCBP'  ]  

final_regrets = {}

for label in labels:

    if label == 'RandCBP':
        result = np.load( './results/benchmark_randcbp/label_efficient/LE_hard_{}_{}_Gaussian_1_10_10e7.npy'.format(horizon,n_folds, label) )
    else: 
        result = np.load( './results/label_efficient/hard_{}_{}_{}.npy'.format(horizon,n_folds, label) )

    final_regrets[label] =  result[:,-1] 

final = np.array( [ (1, i) if final_regrets['RandCBP'][i] <= final_regrets['CBP'][i] else (0, i) for i in range(n_folds) ] )

print( sum(final[:,0]) )

72


In [33]:
### EASY LABEL EFFICIENT:

labels = [  'CBP', 'RandCBP'  ]  

final_regrets = {}

for label in labels:

    if label == 'RandCBP':
        result = np.load( './results/benchmark_randcbp/label_efficient/LE_easy_{}_{}_Gaussian_18_10_10e7.npy'.format(horizon,n_folds, label) )
    else: 
        result = np.load( './results/label_efficient/easy_{}_{}_{}.npy'.format(horizon,n_folds, label) )

    final_regrets[label] =  result[:,-1] 

final = np.array( [ (1, i) if final_regrets['RandCBP'][i] <= final_regrets['CBP'][i] else (0, i) for i in range(n_folds) ] )

print( sum(final[:,0]) )

90


In [ ]:


n_cores = 16
n_folds = 100
horizon = 100000

game = games.label_efficient(  )

#feedexp3.FeedExp3(  game, horizon, ),
#feedexp3_v3.FeedExp3(  game, horizon, ),
#eTSPM.eTSPM_alg(  game, horizon, 1),
#TSPM.TSPM_alg(  game, horizon,), bpm.BPM(  game, horizon,  [0.5, 0.5 ], np.identity(2) ) ,

#TSPM.TSPM_alg(  game, horizon,), 
#ucbTSPM_v2.TSPM_alg(game, horizon)  

algos = [ random_algo.Random(  game, horizon, ), 
          cpb.CPB(  game, horizon, 1.01), 
          None, 
          PM_DMED.PM_DMED(  game, horizon, 100), 
          PM_DMED.PM_DMED(  game, horizon, 10), 
          PM_DMED.PM_DMED(  game, horizon, 5), 
          PM_DMED.PM_DMED(  game, horizon, 1), 
          PM_DMED.PM_DMED(  game, horizon, 0.1), 
          PM_DMED.PM_DMED(  game, horizon, 10), 
          TSPM.TSPM_alg(  game, horizon, 1),
          #TSPM.TSPM_alg(  game, horizon, 0),
          bpm.BPM(game, horizon) ]

colors = [  [0,0,0],  [250,0,0], [0,250,0], [255,128,0], [255,128,0], [255,128,0], [255,128,0],   [0 , 250, 250], [255 , 0, 255] ] 
labels = [   'random', 'CBP', 'RandCBP', 'PM_DMEDc100', 'PM_DMEDc10','PM_DMEDc5','PM_DMEDc1', 'TSPM_R1', 'BPM_LEAST'  ]  

fig = go.Figure( )

final_regrets = []


for alg, color, label in list(zip(algos, colors, labels))[::-1]:

    r,g,b = color
    # result = evaluate_parallel(n_cores, n_folds, horizon, alg, game, 'harsch')
    # np.save('./results/label_efficient/hard_{}_{}_{}'.format(horizon,n_folds, label), result)
    
    if label == 'RandCBP':
        result = np.load( './results/benchmark_randcbp/label_efficient/LE_easy_{}_{}_Gaussian_18_10_10e7.npy'.format(horizon,n_folds, label) )
    else: 
        result = np.load( './results/label_efficient/easy_{}_{}_{}.npy'.format(horizon,n_folds, label) )

    final_regrets.append( result[:,-1] )
    regret =  np.mean(result, 0) 
    xcoords = np.arange(0,horizon,1).tolist()
    std =  np.std(result,0) 
    upper_regret = regret + std

    if label == 'PM_DMEDc100':
        fig.add_trace(go.Scatter(x=xcoords, y=regret, line=dict(color='rgb({},{},{})'.format(r,g,b),  dash=  'dot' ), mode='lines',  name='PMDMED (c=100)' ))  
    elif label == 'PM_DMEDc10':
        fig.add_trace(go.Scatter(x=xcoords, y=regret, line=dict(color='rgb({},{},{})'.format(r,g,b),  dash=  'longdash' ), mode='lines',  name='PMDMED (c=10)' )) 
    elif label == 'PM_DMEDc5':
        fig.add_trace(go.Scatter(x=xcoords, y=regret, line=dict(color='rgb({},{},{})'.format(r,g,b),  dash=  'solid' ), mode='lines',  name='PMDMED (c=5)' )) 
    elif label == 'PM_DMEDc1':
        fig.add_trace(go.Scatter(x=xcoords, y=regret, line=dict(color='rgb({},{},{})'.format(r,g,b),  dash=  'longdashdot' ), mode='lines',  name='PMDMED (c=1)' )) 
    elif label == 'TSPM_R1':
        fig.add_trace(go.Scatter(x=xcoords, y=regret, line=dict(color='rgb({},{},{})'.format(r,g,b),  dash=  'solid' ), mode='lines',  name='TSPM (R=1)' )) 
    elif label == 'TSPM_R0':
        fig.add_trace(go.Scatter(x=xcoords, y=regret, line=dict(color='rgb({},{},{})'.format(r,g,b),  dash=  'dot' ), mode='lines',  name='TSPM (R=0)' )) 
    else:
        fig.add_trace(go.Scatter(x=xcoords, y=regret, line=dict(color='rgb({},{},{})'.format(r,g,b),  dash=  'solid' ), mode='lines',  name=label )) # 

    fig.add_trace(   go.Scatter( x=xcoords+xcoords[::-1], y=upper_regret.tolist()+regret.tolist()[::-1],  fill='toself', fillcolor='rgba({},{},{},0.2)'.format(r,g,b), 
                         line=dict(color='rgba(255,255,255,0)'),   hoverinfo="skip",  showlegend=False )  )
    


fig.update_layout(legend= dict(yanchor="top",y=0.98,xanchor="left",x=0.1), autosize=False,
                  xaxis_title="Sequence", yaxis_title="Regret",  margin=go.layout.Margin( l=0,   r=0,   b=0,    t=0, ),   font=dict(size=13,) )


fig.update_yaxes( type="log",range=[0, 3] )
fig.write_image("./easy_LE.pdf")

fig.update_xaxes( type="log" )
fig.write_image("./easy_LE_log.pdf")

# fig.show()

final_regrets = np.array(final_regrets)
final = [ ( np.argmin(final_regrets[:,i]), i) for i in range(n_folds) ]